# Gathering data from Foundation API (FND Subgraph)

This notebook is gathers data from the Foundation API, which can be accessed here: https://thegraph.com/hosted-service/subgraph/f8n/fnd

In [1]:
import requests
import json
import pandas as pd

In GraphQL, the limit of "first" is [0:1000], the limit of "skip" is [0:5000].  
This leads to a max of 6000 creators from the API.

## NFT Creator Info

In [2]:
query  = """
{
    creators(first: 1, orderBy: netSalesInETH, orderDirection: desc)
    {
    id
    netSalesInETH
    netSalesPendingInETH
    netRevenueInETH
    netRevenuePendingInETH
    }
}
"""

url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
r = requests.post(url, json={'query': query})
json_data = json.loads(r.text)

creator_list = json_data['data']['creators']
df_column_names = list(creator_list[0].keys())
creator_df = pd.DataFrame(columns = df_column_names);

In [3]:
for x in range(6):

    skip_number = x*1000
    
    str1 = "{creators(skip: "
    str2 = str(skip_number)
    str3 = ", first: 1000, orderBy: netSalesInETH, orderDirection: desc){id netSalesInETH netSalesPendingInETH netRevenueInETH netRevenuePendingInETH}}"

    query  = str1 + str2 + str3

    url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
    r = requests.post(url, json={'query': query})
    #print(r.status_code)
    #print(r.text)
    json_data = json.loads(r.text)

    creator_list = json_data['data']['creators']


    for creator in range(len(creator_list)):
        creator_df = creator_df.append(creator_list[creator], ignore_index = True);

In [4]:
creator_df.shape

(6000, 5)

In [5]:
creator_df.describe().T

,count,unique,top,freq
id,6000,6000,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,1
netRevenueInETH,6000,3427,0.85,378
netRevenuePendingInETH,6000,75,0,5869
netSalesInETH,6000,3344,1,380
netSalesPendingInETH,6000,73,0,5869


## NFT sales info (excluding Metadata)

In [6]:
query  = """
{
    creators(first: 1, orderBy: netSalesInETH, orderDirection: desc)
    {
    nfts
        {
        id
        dateMinted
        netSalesInETH
        netSalesPendingInETH
        netRevenueInETH
        netRevenuePendingInETH
        isFirstSale
        percentSplit
        lastSalePriceInETH
        tokenIPFSPath
        }

    }
}
"""

url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
r = requests.post(url, json={'query': query})
json_data = json.loads(r.text)

creator_list = json_data['data']['creators']
df_column_names = list(creator_list[0]['nfts'][0].keys())
nft_sales_df = pd.DataFrame(columns = df_column_names);

In [7]:
for x in range(6):

    skip_number = x*1000
    
    str1 = "{creators(skip: "
    str2 = str(skip_number)
    str3 = ",first: 1000, orderBy: netSalesInETH, orderDirection: desc){nfts{id dateMinted netSalesInETH netSalesPendingInETH netRevenueInETH netRevenuePendingInETH isFirstSale percentSplit lastSalePriceInETH tokenIPFSPath}}}"

    query  = str1 + str2 + str3

    url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
    r = requests.post(url, json={'query': query})
    #print(r.status_code)
    #print(r.text)
    json_data = json.loads(r.text)

    creator_list = json_data['data']['creators']

    for creator in range(len(creator_list)):

        nft_list = creator_list[creator]['nfts']

        for nft in range(len(nft_list)):
            nft_sales_df = nft_sales_df.append(nft_list[nft], ignore_index = True);

In [8]:
nft_sales_df.shape

(58830, 10)

In [9]:
nft_sales_df.describe().T

,count,unique,top,freq
dateMinted,58830,55697,1621941820,10
id,58830,58830,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,1
isFirstSale,58830,2,False,34493
lastSalePriceInETH,34493,2353,1,2708
netRevenueInETH,58830,2906,0,24344
netRevenuePendingInETH,58830,5123,0,46311
netSalesInETH,58830,2580,0,24344
netSalesPendingInETH,58830,5037,0,46311
percentSplit,1987,1,{},1987
tokenIPFSPath,58830,58720,QmPLGMv4YfTXziSQU54CVaxjd1YrvwJcs2RMCXkpC9Y88m...,3


## NFT Metadata Information

In [10]:
tokenIPFSPath_list = nft_sales_df['tokenIPFSPath'].unique()

In [12]:
url_prefix = "https://ipfs.io/ipfs/"

# https://fnd.dev/docs/fnd-subgraph There are 3 metadata fields : "name", "description", "image"
metadata_columns = ["tokenIPFS", "name", "description", "image"]
nft_metadata_df = pd.DataFrame(columns = metadata_columns)

for token in tokenIPFSPath_list:
    url = url_prefix + token
    r = requests.get(url, json={'query': query})
    try:
        json_data = json.loads(r.text)
        json_data["tokenIPFS"] = token
        nft_metadata_df = nft_metadata_df.append(json_data,ignore_index = True)
    except:
        nft_metadata_df = nft_metadata_df.append({"tokenIPFS": token, "name": "","description": "", "image": ""}, ignore_index = True);

In [13]:
nft_metadata_df.shape

(58720, 4)

In [14]:
nft_metadata_df.describe().T

,count,unique,top,freq
tokenIPFS,58720,58720,QmNeqUFf3WKcXFEHgxFCrf3t8jrxyCkev6LBtRz78Ygq9u...,1
name,58720,53547,Serenity,28
description,58720,53438,,1236
image,58720,57790,ipfs://undefined,13


# NFT Bid History Information

In [15]:
query  = """
{
    creators(first: 1, orderBy: netSalesInETH, orderDirection: desc)
    {
    nfts
        {
        nftHistory
            {
            id
            event
            date
            amountInETH
            }
        }

    }
}
"""

url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
r = requests.post(url, json={'query': query})
json_data = json.loads(r.text)

creator_list = json_data['data']['creators']
df_column_names = list(creator_list[0]['nfts'][0]['nftHistory'][0].keys())

nft_bid_df = pd.DataFrame(columns = df_column_names);

In [16]:
#print(r.status_code)
#print(r.text)
for x in range(6):

    skip_number = x*1000
    
    str1 = "{creators(skip: "
    str2 = str(skip_number)
    str3 = ",first: 1000, orderBy: netSalesInETH, orderDirection: desc){nfts{nftHistory{id event date amountInETH}}}}"

    query  = str1 + str2 + str3

    url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
    r = requests.post(url, json={'query': query})
    #print(r.status_code)
    #print(r.text)
    json_data = json.loads(r.text)

    creator_list = json_data['data']['creators']

    for creator in range(len(creator_list)):

        nft_list = creator_list[creator]['nfts']

        for nft in range(len(nft_list)):

            nft_bid_list = nft_list[nft]['nftHistory']

            for nft_bid in range(len(nft_bid_list)):

                nft_bid_df = nft_bid_df.append(nft_bid_list[nft_bid], ignore_index = True);
        

In [17]:
nft_bid_df.shape

(327619, 4)

In [18]:
nft_bid_df.describe().T

,count,unique,top,freq
amountInETH,209491,5587,1,18532
date,327619,275340,1631220937,42
event,327619,14,Bid,81929
id,327619,327619,0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd...,1


## Connector 1

In [19]:
df_column_names = ['creator_id','nft_id']
creator_nft_df = pd.DataFrame(columns = df_column_names)

In [20]:
for x in range(6):

    skip_number = x*1000
    
    str1 = "{creators(skip: "
    str2 = str(skip_number)
    str3 = ",first: 1000, orderBy: netSalesInETH, orderDirection: desc){id nfts{id}}}"

    query  = str1 + str2 + str3

    url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
    r = requests.post(url, json={'query': query})
    #print(r.status_code)
    #print(r.text)
    json_data = json.loads(r.text)

    creator_dict = json_data['data']['creators']

    for creator_number in range(len(creator_dict)):

        creator_list = list()
        creator_str = json_data['data']['creators'][creator_number]['id']
        creator_list += [creator_str]
        creator_df_temp = pd.DataFrame(creator_list, columns = ['creator_id'])


        nft_data = json_data['data']['creators'][creator_number]['nfts']
        nft_list = list()
        for nft_number in range(len(nft_data)):
            nft_str = json_data['data']['creators'][creator_number]['nfts'][nft_number]['id']
            nft_list += [nft_str]

        nft_df = pd.DataFrame(nft_list,columns = ['nft_id'])


        combined_df = pd.concat([creator_df_temp, nft_df], axis = 1)
        combined_df.fillna(method='ffill', inplace = True)

        creator_nft_df = pd.concat([creator_nft_df, combined_df])

    creator_nft_df.reset_index;    

In [21]:
creator_nft_df.shape

(58904, 2)

In [22]:
creator_nft_df.describe().T

,count,unique,top,freq
creator_id,58904,6000,0xd4518a94adc2fcaf1478e7c5ccc029be0f61ced9,100
nft_id,58871,58871,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,1


## Connector 2

In [23]:
df_column_names = ['nft_id','nft_history_id']
nft_nft_history_df = pd.DataFrame(columns = df_column_names)

In [24]:
for x in range(6):

    skip_number = x*1000
    
    str1 = "{creators(skip: "
    str2 = str(skip_number)
    str3 = ",first: 1000, orderBy: netSalesInETH, orderDirection: desc){nfts{id nftHistory{id}}}}"

    query  = str1 + str2 + str3

    url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
    r = requests.post(url, json={'query': query})
    # convert json into list
    json_data = json.loads(r.text)

    creator_data = json_data['data']['creators']

    for creator in range(len(creator_data)):

        nft_data = creator_data[creator]['nfts']

        for nft in range(len(nft_data)):
            nft_str = nft_data[nft]['id']
            nft_list = [nft_str]

            nft_history_data = nft_data[nft]['nftHistory']
            nft_history_list = list ()
            for nft_history in range(len(nft_history_data)):
                nft_history_str = nft_history_data[nft_history]['id']
                nft_history_list += [nft_history_str]

            nft_history_df = pd.DataFrame(nft_history_list, columns = ['nft_history_id'])
            nft_df = pd.DataFrame(nft_list, columns = ['nft_id'])
            combined_df = pd.concat([nft_df,nft_history_df],axis = 1)
            combined_df.fillna(method='ffill', inplace = True)

            nft_nft_history_df = pd.concat([nft_nft_history_df,combined_df])

    nft_nft_history_df.reset_index;

In [25]:
nft_nft_history_df.shape

(327651, 2)

In [26]:
nft_nft_history_df.describe().T

,count,unique,top,freq
nft_id,327651,58871,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-46856,54
nft_history_id,327651,327651,0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd...,1


## storing the data 

In [27]:
datapath = "../data/raw/"
creator_df.to_csv(datapath + 'creator_df.csv', index = False)
nft_sales_df.to_csv(datapath + 'nft_sales_df.csv', index = False)
nft_metadata_df.to_csv(datapath + 'nft_metadata_df.csv', index = False)
nft_bid_df.to_csv(datapath + 'nft_bid_df.csv', index = False)
creator_nft_df.to_csv(datapath + 'creator_nft_df.csv', index = False)
nft_nft_history_df.to_csv(datapath + 'nft_nft_history_df.csv', index = False)